In [ ]:
# 나눔글꼴 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

나눔글꼴을 설치한 후 코랩 `런타입 다시 시작(`Ctrl+M .`)`을 실행하고 아래 코드 셀부터 이어서 실습을 진행


In [ ]:
# 그래프 기본 글꼴 설정

import matplotlib.pyplot as plt

font = 'NanumBarunGothic'
plt.rc('font', family=font)

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive/data-analytics-101/data/'

# 여덟째마당. 텍스트 마이닝

- **텍스트 마이닝(text mining)**: 문자로된 데이터에서 가치 있는 정보를 얻어내는 데이터 분석 기법
- **형태소 분석(morphoplogy analysis)**: 문장을 구성하는 어절들이 어떤 품사인지 분석하는 과정. 어절의 품사를 파악한 다음 명사, 동사, 형용사 등 의미를 지닌 품사를 추출해 어떤 단어가 얼마나 많이 사용됐는지 확인.

## 대통령 후보자 연설문 텍스트 마이닝

대통령 연설문은 문법 오류가 적고 정제된 문장으로 되어 있어 전처리 작업이 적어 텍스트 마이닝을 익히는 데 매우 적합함.

### `KoNLPy` 패키지 설치하기

`KoNLPy` 패키지를 사용하면 한글 텍스트에 대해 형태소 분석을 할 수 있음.

In [ ]:
!pip install konlpy

### 가장 많이 사용된 단어 알아보기

#### 1. 연설문 불러오기

`UTF-8`으로 인코딩된 텍스트 파일 불러오기

* 인코딩(encoding): 컴퓨터가 문자를 표현하는 방식으로 문서마다 인코딩 방식이 다르기 때문에 인코딩이 맞지 않으면 글자가 깨져 보일 수 있음

In [ ]:
speech = open(path + 'speech_moon.txt', encoding = 'UTF-8').read()
speech

#### 2. 불필요한 문자 제거하기

출력 결과를 보면 한글 외에 특수문자, 한자, 공백 등 분석 대상이 아닌 문자들이 많이 있기 때문에, 정규표현식(regular expression)으로 문자열 치환을 처리할 수 있는 파이썬 내부 모듈 `re`의 `sub` 함수를 사용하여 한글이 아닌 모든 문자를 공백(' ')으로 대치

In [ ]:
# 불필요한 문자 제거하기
import re
speech = re.sub('[^가-힣]', ' ', speech)
speech

#### 3. 명사 추출하기

In [ ]:
# Hannanum(한나눔) 형태소 분석기 객체 만들기

import konlpy
hannanum = konlpy.tag.Hannanum()

In [ ]:
# 명사 추출하기
hannanum.nouns("대한민국의 영토는 한반도와 그 부속도서로 한다")

In [ ]:
# 연설문에서 명사 추출하기
nouns = hannanum.nouns(speech)
nouns

In [ ]:
# 데이터 프레임으로 변환
import pandas as pd

df_word = pd.DataFrame({'word' : nouns})
df_word

#### 4. 단어 빈도표 만들기

##### 한 글자로된 단어 제거하기

`pd.str.len()`을 이용하면 글자 수를 계산할 수 있음

In [ ]:
# 글자 수 추가
df_word['len'] = df_word['word'].str.len()
df_word

In [ ]:
df_word.sort_values('len', ascending=False)

In [ ]:
# 두 글자 이상 단어만 남기기
df_word = df_word.query('len >= 2')
df_word.sort_values('len', ascending=False)

##### 단어 빈도 구하기

In [ ]:
df_word.groupby('word', as_index=False).agg(n = ('word', 'count')).sort_values('n', ascending=False)

In [ ]:
# 단어별 분리
# 빈도 구하기
# 내림차순 정렬
df_word = df_word.groupby('word', as_index = False) \
                 .agg(n = ('word', 'count')) \
                 .sort_values('n', ascending = False)
df_word

#### 5. 단어 빈도 막대 그래프 만들기

In [ ]:
# 단어 빈도 상위 20개 추출
top20 = df_word.head(20)
top20

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams.update({'font.family'    : font,       # 한글 폰트 설정
                     'figure.dpi'     : '100',      # 해상도 설정
                     'figure.figsize' : [6, 6]})    # 가로 세로 크기 설정

# 막대 그래프 만들기
sns.barplot(data = top20, y = 'word', x = 'n')

### 워드 클라우드 만들기

`워크 클라우드(word cloud)'는 단어의 빈도를 구름 모양으로 표현한 그래프로, 단어의 빈도에 따라 글자의 크기나 색깔을 다르게 표현해서 어떤 단어가 얼마나 많이 사용됐는지 한 눈에 파악할 수 있음

#### 1. `wordcloud` 패키지 설치하기

In [ ]:
!pip install wordcloud

#### 2. 단어와 빈도를 담은 딕셔너리 만들기

In [ ]:
df_word.set_index('word').to_dict()['n']

In [ ]:
# 데이터 프레임을 딕셔너리로 변환
dic_word = df_word.set_index('word').to_dict()['n']
dic_word

#### 3. 워드 클라우드 만들기

In [ ]:
# WordCloud 객체 만들기
from wordcloud import WordCloud

wc = WordCloud(random_state = 42,           # 난수 고정
               font_path = font,            # 폰트 설정
               width = 400,                 # 가로 크기
               height = 400,                # 세로 크기
               background_color = 'white')  # 배경색

In [ ]:
# 워드 클라우드를 생성하고 출력하기
img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.imshow(img_wordcloud)
plt.axis('off')
plt.show()

### 워드 클라우드 모양 바꾸기

#### 1. mask 만들기

In [ ]:
import PIL
import numpy as np

icon = PIL.Image.open(path + 'cloud.png')

img = PIL.Image.new('RGB', icon.size, (255, 255, 255))
img.paste(icon, icon)
img = np.array(img)

#### 2. 워드 클라우드 만들기

In [ ]:
# wc 만들기
wc = WordCloud(random_state = 42,
               font_path = font,            # 폰트 설정
               width = 400,                 # 가로 크기
               height = 400,                # 세로 크기
               background_color = 'white',  # 배경색
               mask = img)                  # mask 설정

In [ ]:
# 워드 클라우드를 생성하고 출력하기
img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.imshow(img_wordcloud)
plt.axis('off')
plt.show()

### 워드 클라우드 색깔 바꾸기

In [ ]:
# wc 만들기
wc = WordCloud(random_state = 42,
               font_path = font,            # 폰트 설정
               width = 400,                 # 가로 크기
               height = 400,                # 세로 크기
               background_color = 'white',  # 배경색
               mask = img,                  # mask 설정
               colormap = 'inferno')        # 컬러맵 설정

In [ ]:
# 워드 클라우드를 생성하고 출력하기
img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.imshow(img_wordcloud)
plt.axis('off')
plt.show()

## 기사 댓글 텍스트 마이닝 (feat. BTS)

### 가장 많이 사용된 단어 알아보기

#### 1. 기사 댓글 불러오기

In [ ]:
df = pd.read_csv(path + 'news_comment_BTS.csv', encoding = 'UTF-8')

df.info()

#### 2. 불필요한 문자 제거하기

In [ ]:
df['reply'] = df['reply'].str.replace('[^가-힣]', ' ', regex = True)
df['reply'].head()

#### 3. 명사 추출하기

In [ ]:
# 꼬꼬마(kkma) 형태소 분석기 객체 만들기
import konlpy
kkma = konlpy.tag.Kkma()

In [ ]:
# 명사 추출 - df.apply() 활용

nouns = df['reply'].apply(kkma.nouns)
nouns

#### 4. 단어 빈도표 만들기

In [ ]:
# 리스트를 분해해서 한 행에 한 단어만 들어가도록 수정
nouns = nouns.explode()
nouns

In [ ]:
# 데이터 프레임 만들기
df_word = pd.DataFrame({'word' : nouns})
df_word

In [ ]:
# 글자 수 추가
df_word['len'] = df_word['word'].str.len()
df_word.head()

In [ ]:
# 두 글자 이상 단어만 남기기
df_word = df_word.query('len >= 2')   # (('len >= 2') | ('word not in ["방탄", "소년단"]'))
df_word.head()

In [ ]:
# 단어별 분리
# 빈도 구하기
# 내림차순 정렬
df_word = df_word.groupby('word', as_index = False) \
                 .agg(n = ('word', 'count')) \
                 .sort_values('n', ascending = False)
df_word

#### 5. 단어 빈도 막대 그래프 만들기

In [ ]:
# 단어 빈도 상위 20개 추출
top20 = df_word.head(20)
top20

In [ ]:
# 막대 그래프 만들기
sns.barplot(data = top20, y = 'word', x = 'n')

### 워드 클라우드 만들기

In [ ]:
# 데이터 프레임을 딕셔너리로 변환
dic_word = df_word.set_index('word').to_dict()['n']

In [ ]:
# wc 만들기
wc = WordCloud(random_state = 42,           # 난수 고정
               font_path = font,            # 폰트 설정
               width = 400,                 # 가로 크기
               height = 400,                # 세로 크기
               background_color = 'white',  # 배경색
               mask = img)                  # mask 설정

In [ ]:
# 워드 클라우드 만들기
img_wordcloud = wc.generate_from_frequencies(dic_word)

# 워드 클라우드 출력하기
plt.imshow(img_wordcloud) 
plt.axis('off')                 # 테두리 선 없애기
plt.show()